In [ ]:
#xgb.train: 
    # early_stopping_rounds = 10
        # "Early stopping can be enabled by the parameter early_stopping_rounds. Early stopping can help prevent overfitting and save time during training."
    # num_boost_round = 1000
    # params
#xgb.XGBClassifier
# XGBClassifier follows the scikit-learn classifier API
    # QSTN: are params the same as xgb train? I see early_stopping_rounds
    # tree_method="hist" (QSTN: when to use hist or other methods?)




#QSTN: diff classes
#QSTN: setting the right parameters
# TODO: handle categorical data

#TODO: make sure to account for categorical data correctly.

# https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html
# There are in general two ways that you can control overfitting in XGBoost:
    # The first way is to directly control model complexity. This includes max_depth, min_child_weight and gamma.
    # The second way is to add randomness to make training robust to noise. This includes subsample and colsample_bytree. You can also reduce stepsize eta. Remember to increase num_round when you do so.




In [ ]:
#  With the sklearn estimator interface, we can train a classification model with only a couple lines of Python code. Here’s an example for training a classification model:


In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import xgboost as xgb

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=94)

# Use "hist" for constructing the trees, with early stopping enabled.
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
# Fit the model, test sets are used for early stopping.
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
# At present, XGBoost doesn’t implement data spliting logic within the estimator and relies on the eval_set parameter of the xgboost.XGBModel.fit() method. If you want to use early stopping to prevent overfitting, you’ll need to manually split your data into training and testing sets using the sklearn.model_selection.train_test_split() function from the sklearn library.

# Save model into JSON format.
clf.save_model("clf.json")

[0]	validation_0-logloss:0.45997
[1]	validation_0-logloss:0.34184
[2]	validation_0-logloss:0.27076
[3]	validation_0-logloss:0.22399
[4]	validation_0-logloss:0.19346
[5]	validation_0-logloss:0.16814
[6]	validation_0-logloss:0.15393
[7]	validation_0-logloss:0.14081
[8]	validation_0-logloss:0.13269
[9]	validation_0-logloss:0.12515
[10]	validation_0-logloss:0.11551
[11]	validation_0-logloss:0.11184
[12]	validation_0-logloss:0.10799
[13]	validation_0-logloss:0.10541
[14]	validation_0-logloss:0.10493
[15]	validation_0-logloss:0.10326
[16]	validation_0-logloss:0.10300
[17]	validation_0-logloss:0.10339
[18]	validation_0-logloss:0.10160
[19]	validation_0-logloss:0.09892
[20]	validation_0-logloss:0.09478
[21]	validation_0-logloss:0.09359
[22]	validation_0-logloss:0.09247
[23]	validation_0-logloss:0.09284


In [ ]:
# Some other machine learning algorithms, like those in sklearn, include early stopping as part of the estimator and may work with cross validation. However, using early stopping during cross validation may not be a perfect approach because it changes the model’s number of trees for each validation fold, leading to different model. A better approach is to retrain the model after cross validation using the best hyperparameters along with early stopping. If you want to experiment with idea of using cross validation with early stopping, here is a snippet to begin with:

from sklearn.base import clone
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, cross_validate

import xgboost as xgb

X, y = load_breast_cancer(return_X_y=True)


def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=94)

clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=3)

resutls = {}

for train, test in cv.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    est, train_score, test_score = fit_and_score(
        clone(clf), X_train, X_test, y_train, y_test
    )
    resutls[est] = (train_score, test_score)

# n-jobs and threads:
# If both XGBoost and sklearn are set to use all threads, your computer may start to slow down significantly due to something called “thread thrashing”. To avoid this, you can simply set the n_jobs parameter for XGBoost to None (which uses all threads) and the n_jobs parameter for sklearn to 1. This way, both programs will be able to work together smoothly without causing any unnecessary computer strain.

In [10]:
# https://xgboost.readthedocs.io/en/stable/python/examples/sklearn_examples.html
import pickle

import numpy as np
from sklearn.datasets import fetch_california_housing, load_digits, load_iris
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split

import xgboost as xgb

rng = np.random.RandomState(31337)

print("Zeros and Ones from the Digits dataset: binary classification")
digits = load_digits(n_class=2)
y = digits['target']
X = digits['data']
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBClassifier(n_jobs=1).fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(confusion_matrix(actuals, predictions))

print("Iris: multiclass classification")
iris = load_iris()
y = iris['target']
X = iris['data']
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBClassifier(n_jobs=1).fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(confusion_matrix(actuals, predictions))

print("California Housing: regression")
X, y = fetch_california_housing(return_X_y=True)
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBRegressor(n_jobs=1).fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(mean_squared_error(actuals, predictions))

print("Parameter optimization")
xgb_model = xgb.XGBRegressor(n_jobs=1)
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2, 4],
                    'n_estimators': [50, 100]}, verbose=1, n_jobs=1, cv=3)
clf.fit(X, y)
print(clf.best_score_)
print(clf.best_params_)
xgb_model = xgb.XGBRegressor(max_depth = 4, n_estimators = 100)
xgb_model.fit(X, y)
predictions = xgb_model.predict(X[test_index])
actuals = y[test_index]
print(mean_squared_error(actuals, predictions))


Zeros and Ones from the Digits dataset: binary classification
[[87  0]
 [ 1 92]]
[[91  0]
 [ 3 86]]
Iris: multiclass classification
[[19  0  0]
 [ 1 31  2]
 [ 0  3 19]]
[[31  0  0]
 [ 0 16  0]
 [ 0  1 27]]
California Housing: regression
0.2310805601814026
0.23533764411819863
Parameter optimization
Fitting 3 folds for each of 4 candidates, totalling 12 fits
0.6838972456687431
{'max_depth': 4, 'n_estimators': 100}


/usr/local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [15:15:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "param" } are not used.

  warnings.warn(smsg, UserWarning)


0.08224154667269369
